In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [3]:
import pandas as pd

columns=['source_id', 'source_name', 'composer', 'work', 'country', 'state', 'city', 'theatre', 
         'date_start', 'date_end', 'premiere_date', 'performance_count', 'composer_country', 'theatre_category']

cursor = conn.cursor()  
query = "SELECT s.id as source_id, " + \
        "       s.name as source_name, " + \
        "       coalesce(c.name, l.composer) as composer, " + \
        "       coalesce(w.name, l.work) as work, " + \
        "       l.country, " + \
        "       l.state, " + \
        "       l.city, " + \
        "       l.theatre, " + \
        "       l.date_start, " + \
        "       l.date_end, " + \
        "       w.premiere_date, " + \
        "       l.performance_count, " + \
        "       c.country as composer_country, " + \
        "       CASE WHEN l.source_id = 1 then 'Professional' when opera_america_name IS NULL THEN 'Amateur' ELSE 'Professional' END as theatre_category " + \
        "from sources s " + \
        "INNER JOIN listings l on s.id = l.source_id " + \
        "LEFT JOIN theatre_mapping tm ON l.theatre = tm.name " + \
        "LEFT JOIN listing_mappers lm ON l.map_key = lm.map_key " + \
        "LEFT JOIN works w ON lm.work_uri = w.uri " + \
        "LEFT JOIN composers c ON w.composer_id = c.id " + \
        "WHERE w.uri != 'W1' " + \
        "ORDER BY s.id DESC, l.country, l.state, l.city"
        
cursor.execute(query)

mdf = pd.DataFrame(cursor.fetchall(), columns=columns)
mdf['year'] = mdf.apply(lambda row: row['date_start'].year, axis=1)
mdf['decade'] = mdf.apply(lambda row: 10*int(row['year']/10), axis=1)
mdf[['premiere_date']] = mdf[['premiere_date']].fillna(value=0)
mdf['work_decade'] = mdf.apply(lambda row: 10*int(row['premiere_date']/10), axis=1)


In [4]:
######################## libraries ########################

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as ticker
import numpy as np

import sys
sys.path.append('../../')

from utils.utility_functions import label_top
from utils.utility_functions import get_last_name
from utils.utility_functions import make_dash_zero

##################### CHARTING OPTIONS #####################

# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Palatino']})

rc('text', usetex=True)
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

colors = ['#8a170f', '#8a540f', '#828a0f', '#458a0f', '#0f8a17', '#0f8a54', '#0f828a', '#0f458a', '#170f8a', '#540f8a', '#8a0f82', '#8a0f45']


##################### SOURCES OPTIONS #####################

archival_sources = ['Metropolitan Opera', 'Opera Scotland', 'Royal Opera House', 'Teatro alla Scala', 'Wiener Staatsoper']
commercial_sources_world = ['bachtrack.com','operabase.com','operacritic.com']
commercial_sources_uk = ['Concert-Diary','U.K. Theatre Web']
government_sources = ['National Central Opera Service Reports', 'Opera America']

##############################################################################

In [25]:
######################## WORLD           ########################
######################## DoW per country ########################

o_df = mdf.copy()
o_df = o_df[o_df['source_name'].isin(commercial_sources_world)]
o_df = o_df[o_df['year'] > 2007]
o_df = o_df[['composer', 'work', 'country', 'state', 'city', 'date_start', 'date_end', 'premiere_date', 'performance_count', 'year', 'decade', 'work_decade', 'composer_country']]
o_df = o_df.drop_duplicates()
o_df['weekday'] = o_df.apply(lambda row: row['date_start'].isoweekday(), axis=1)

countries_excl = ['Oman','Macedonia','Armenia','South Korea','Egypt','Georgia','Albania','Kyrgyzstan','Mongolia','Singapore',
                  'Bosnia-Herzegovina','Malta','Iceland','Mexico','China','Thailand','Peru','Costa Rica','India','UAE']

o_df = o_df[~o_df['country'].isin(countries_excl)]
countries = o_df['country'].unique().tolist()
countries.sort()

o_df = o_df.groupby(['country', 'weekday'])
o_df = o_df.agg({'performance_count': np.sum })
o_df = o_df.groupby(level = ['country']).transform(lambda x: x/x.sum())

for num, country in enumerate(countries):
    c_df = o_df.loc[country]
    c_df = c_df.reset_index(level=['weekday'])        
    c_df = c_df.rename({'performance_count': country}, axis='columns')    
#     print(c_df.head())
    c_df = c_df.set_index('weekday')
    c_df = c_df[country]
    
    if num==0:
        df = c_df.copy().to_frame()
    else:
        df = df.join(c_df, how='outer')
        
df = df.T
df = df.fillna(value=0)
df      

DoW = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
df = df.rename(DoW, axis='columns')    


#For displaying as rows
decades = df.columns.get_values().tolist()
decades = [str(d) for d in decades]
decades = ['\\textbf{' + d + '}' for d in decades]

print('l'*len(decades))
print('\\textbf{Country} & ' + ' & '.join(decades) + ' \\\ \midrule ')    
for idx,row in df.iterrows():
    values = row.values.tolist()
    values = [str(int(100*c)) for c in values ]    
    print(str(idx) + ' & ' + ' & '.join(values) + " \\\\")

###############################################################

lllllll
\textbf{Country} & \textbf{Monday} & \textbf{Tuesday} & \textbf{Wednesday} & \textbf{Thursday} & \textbf{Friday} & \textbf{Saturday} & \textbf{Sunday} \\ \midrule 
Argentina & 1 & 21 & 6 & 8 & 20 & 17 & 24 \\
Australia & 4 & 15 & 14 & 16 & 14 & 27 & 6 \\
Austria & 8 & 10 & 14 & 14 & 16 & 18 & 16 \\
Azerbaijan & 0 & 1 & 24 & 4 & 3 & 38 & 26 \\
Belarus & 4 & 24 & 8 & 23 & 6 & 19 & 12 \\
Belgium & 0 & 20 & 12 & 13 & 17 & 13 & 21 \\
Bulgaria & 8 & 12 & 13 & 15 & 23 & 15 & 11 \\
Canada & 1 & 14 & 9 & 15 & 14 & 29 & 13 \\
Chile & 16 & 15 & 12 & 15 & 17 & 20 & 2 \\
Croatia & 15 & 10 & 15 & 14 & 17 & 20 & 5 \\
Czech Republic & 5 & 14 & 14 & 14 & 16 & 17 & 15 \\
Denmark & 6 & 14 & 16 & 12 & 14 & 19 & 14 \\
Estonia & 0 & 4 & 16 & 20 & 19 & 23 & 15 \\
Finland & 6 & 13 & 17 & 18 & 20 & 22 & 1 \\
France & 6 & 17 & 11 & 12 & 18 & 13 & 20 \\
Germany & 3 & 7 & 10 & 12 & 18 & 23 & 24 \\
Greece & 0 & 5 & 19 & 7 & 20 & 17 & 28 \\
Hungary & 4 & 9 & 11 & 14 & 17 & 21 & 21 \\
Ireland & 7 & 12 & 12 &